https://nbviewer.jupyter.org/github/yitzchak/common-lisp-jupyter/blob/master/examples/about.ipynb

https://nbviewer.jupyter.org/github/yitzchak/common-lisp-jupyter/blob/master/examples/widgets.ipynb

https://nbviewer.jupyter.org/github/yitzchak/common-lisp-jupyter/blob/master/examples/julia.ipynb

In [1]:
(format t "Hello, World")
(format *error-output* "Goodbye, cruel World.")

NIL

NIL

Hello, World

Goodbye, cruel World.

In [2]:
(+ 2 3 4 5)

14

In [13]:
(ql:quickload :jsown)

To load "jsown":
  Load 1 ASDF system:
    jsown


(:JSOWN)


; Loading "jsown"



In [18]:
(jsown:to-json
    (jsown:new-js ("foo" "bar")))

"{\"foo\":\"bar\"}"

In [15]:
(defun fibonacci (n)
    (if (<= n 1)
        1
        (+ (fibonacci (- n 2)) (fibonacci (- n 1)))))


(fibonacci 7)

FIBONACCI

21


REDEFINITION-WITH-DEFUN: 
  redefining COMMON-LISP-USER::FIBONACCI in DEFUN


In [16]:
(jupyter:latex "$$R_{\\mu \\nu} - \\tfrac{1}{2}R \\, g_{\\mu \\nu} + \\Lambda g_{\\mu \\nu} =
               8 \\pi G c^{-4} T_{\\mu \\nu}$$")

inline-value

In [19]:
(ql:quickload '(:alexandria :png :iterate :flexi-streams))
(use-package :alexandria)
(use-package :iterate)
(use-package :jupyter-widgets)

To load "alexandria":
  Load 1 ASDF system:
    alexandria

; Loading "alexandria"

To load "png":
  Load 3 ASDF systems:
    asdf cffi cffi-grovel
  Install 1 Quicklisp release:
    cl-png

; Fetching #<URL "http://beta.quicklisp.org/archive/cl-png/2019-07-10/cl-png-vl-anyversion-04dbb706-git.tgz">
; 8311.06KB

8,510,528 bytes in 1.43 seconds (5824.15KB/sec)

; Loading "png"
[package image]...................................
[package png].; cc -o /home/rpo/.cache/common-lisp/sbcl-1.3.1.debian-linux-x64/home/rpo/quicklisp/dists/quicklisp/software/cl-png-vl-anyversion-04dbb706-git/grovel__grovel-tmp6TLO0BDD.o -c -m64 -I/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX10.7.sdk/usr/X11/include -I/opt/local/include -fPIC -I/home/rpo/quicklisp/dists/quicklisp/software/cffi_0.22.1/ /home/rpo/.cache/common-lisp/sbcl-1.3.1.debian-linux-x64/home/rpo/quicklisp/dists/quicklisp/software/cl-png-vl-anyversion-04dbb706-git/grovel__grovel.c
; cc -o /home/rpo/.c

(:ALEXANDRIA :PNG :ITERATE :FLEXI-STREAMS)

T

T

T

In [20]:
(defun julia-count (z c &key (max 255))
    (iter
        (for i from 0 to (1- max))
        (for p first z then (+ (expt p 2) c))
        (while (<= (abs p) 2))
        (finally (return i))))

JULIA-COUNT

In [21]:
(defclass julia-widget ()
    ((image :reader julia-image
            :initform (make-instance 'image :width 640 :height 640))
     (x :reader julia-x
        :initform (make-instance 'float-text :value 0 :description "x"))
     (y :reader julia-y
        :initform (make-instance 'float-text :value 0 :description "y"))
     (size :reader julia-size
           :initform (make-instance 'float-text :value 4 :description "size"))
     (ca :reader julia-ca
         :initform (make-instance 'float-text :value -0.8 :step 0.001 :description "ca"))
     (cb :reader julia-cb
         :initform (make-instance 'float-text :value 0.156 :step 0.001 :description "cb"))
     (progress :reader julia-progress
               :initform (make-instance 'int-progress :max 640 :description "Complete"))
     (controls :reader julia-controls
               :initform (make-instance 'v-box))
     (container :reader julia-container
                :initform (make-instance 'h-box))))

(defvar jw (make-instance 'julia-widget))

(defparameter *update-thread* nil)

(defun make-julia (image progress c &key (width 100) (height 100) (xmin -2.5) (xmax 2.5) (ymin -2.5) (ymax 2.5))
    (let ((img (png:make-image height width 1 8)))
        (iter
            (for x from 0 to (1- width))
            (for zx next (+ xmin (* (coerce (/ x width) 'float) (- xmax xmin))))
            (setf (widget-value progress) x)
            (iter
                (for y from 0 to (1- height))
                (for zy next (+ ymin (* (coerce (/ y height) 'float) (- ymax ymin))))
                (for j next (julia-count (complex zx zy) c))
                (setf (aref img y x 0) (- 255 j))))
        (setf (widget-value image) (flexi-streams:with-output-to-sequence (o)
            (png:encode img o)))
        nil))

(defun update ()
    (when (and *update-thread* (bordeaux-threads:thread-alive-p *update-thread*))
        (bordeaux-threads:destroy-thread *update-thread*))
    (setq *update-thread* (bordeaux-threads:make-thread
        (lambda ()
            (with-slots (image x y size ca cb progress) jw
                (when-let ((ca-value (widget-value ca))
                           (cb-value (widget-value cb))
                           (x-value (widget-value x))
                           (y-value (widget-value y))
                           (size-value (widget-value size)))
                    (make-julia image progress (complex ca-value cb-value)
                                :width (widget-width image) :height (widget-height image)
                                :xmin (- x-value (/ size-value 2))
                                :xmax (+ x-value (/ size-value 2))
                                :ymin (- y-value (/ size-value 2))
                                :ymax (+ y-value (/ size-value 2)))))))))
                    

(with-slots (image x y size ca cb controls container progress) jw
    (setf (widget-children controls) (list x y size ca cb progress))
    (setf (widget-children container) (list image controls))
    (defmethod jupyter-widgets:on-trait-change :after ((object (eql x)) type name old-value new-value source)
        (update))
    (defmethod jupyter-widgets:on-trait-change :after ((object (eql y)) type name old-value new-value source)
        (update))
    (defmethod jupyter-widgets:on-trait-change :after ((object (eql size)) type name old-value new-value source)
        (update))
    (defmethod jupyter-widgets:on-trait-change :after ((object (eql ca)) type name old-value new-value source)
        (update))
    (defmethod jupyter-widgets:on-trait-change :after ((object (eql cb)) type name old-value new-value source)
        (update))
    (update))

#<STANDARD-CLASS COMMON-LISP-USER::JULIA-WIDGET>

JW

*UPDATE-THREAD*

MAKE-JULIA

UPDATE

#<SB-THREAD:THREAD "Anonymous thread" RUNNING {1006DA5123}>

In [23]:
(julia-container jw)

A Jupyter Widget